In [1]:
import pandas as pd
import numpy as np
from time import time

import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

import matplotlib.pyplot as plt
from math import pi

from omterms.interface import *

import pickle

from ipywidgets import interact, fixed

import requests
import json

## Plots and Prints

In [2]:
categories=['universalism', 'hedonism', 'achievement', 'power',
       'self-direction', 'benevolence', 'conformity', 'tradition', 'stimulation',
       'security']

def plot_radar_chart(doc_topic_cumul, doc):
    # ------- PART 1: Create background
 
    # number of variablecategories
    schwartz =['universalism', 'benevolence', 'conformity', 'tradition',
       'security', 'power', 'achievement', 'hedonism', 'stimulation',
       'self-direction']
    
    schwartz_dist = []
    for sch in schwartz:
        schwartz_dist.append(doc_topic_cumul[doc][categories.index(sch)])
    
    N = len(schwartz)
    
    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    plt.figure(figsize=(8,8))
    # Initialise the spider plot
    ax = plt.subplot(111, polar=True)

    # If you want the first axis to be on top:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)

    # Draw one axe per variable + add labels labels yet
    plt.xticks(angles[:-1], schwartz)

    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks([25,50,75], ["25","50","75"], color="grey", size=7)
    plt.ylim(0,100)


    # ------- PART 2: Add plots

    # Plot each individual = each line of the data
    # I don't do a loop, because plotting more than 3 groups makes the chart unreadable

    # Ind1
    values = list(schwartz_dist) + list(schwartz_dist[:1])
    ax.plot(angles, values, linewidth=1, linestyle='solid')
    ax.fill(angles, values, 'b', alpha=0.1)

    # Add legend
    #plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.title("Schwartz Chart - Doc " + str(doc))
    plt.show()
    
    
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
    
    
def print_top_words(model, tfidf_vectorizer, n_top_words, n_topics=3):
    feature_names = tfidf_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        if topic_idx % n_topics == 0:
            try:
                print(color.CYAN + color.BOLD + categories[topic_idx//3] + color.END)
            except:
                print(color.CYAN + color.BOLD + "General" + color.END)
        message = color.BOLD + "Topic #%d: " % topic_idx + color.END
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
        if (topic_idx+1) % n_topics == 0:
            print()
    print()
    
def print_cumulative_train_doc_topics(data, doc_topic, doc, n_best):
    test_theme = data.iloc[doc]['theme']
    print(color.BOLD + "Doc " + str(doc) + color.RED +  " (" + test_theme + ")\t: " + color.END, end='')
    dt = doc_topic[doc]
    for i in dt.argsort()[:-n_best - 1:-1]:
        print("(", end='')
        try:
            print(color.CYAN + color.BOLD + categories[i] + color.END, end='')
        except:
            print(color.CYAN + color.BOLD + "General" + color.END, end='')
        print(", %d, %.2lf)  " %(i, dt[i]), end='')    
    print()
    
def print_cumulative_test_doc_topics(doc_topic, doc, n_best):
    print(color.BOLD + "Doc " + str(doc) + "\t: " + color.END, end='')
    dt = doc_topic[doc]
    for i in dt.argsort()[:-n_best - 1:-1]:
        print("(", end='')
        try:
            print(color.CYAN + color.BOLD + categories[i] + color.END, end='')
        except:
            print(color.CYAN + color.BOLD + "General" + color.END, end='')
        print(", %d, %.2lf)  " %(i, dt[i]), end='')    
    print()

def print_doc_topics(doc_topic, doc, n_best):
    print(color.BOLD + "Doc " + str(doc) + "\t: " + color.END, end='')
    for i in doc_topic[doc].argsort()[:-n_best - 1:-1]:
        print("(", end='')
        try:
            print(color.CYAN + color.BOLD + categories[i//3] + color.END, end='')
        except:
            print(color.CYAN + color.BOLD + "General" + color.END, end='')
        print(", %d, %.2lf)  " %(i, doc_topic[doc][i]), end='')    
    print()

def print_train_results(doc_topic, doc, corpus, data):
    print(color.BOLD + "Document " + str(doc) + color.END)
    print()
    print(color.BOLD + "Text: " + color.END)
    print("..." + corpus[doc][len(corpus[doc])//3:len(corpus[doc])//3+500] + "...")
    print()
    print()
    
    print(color.BOLD + "Topic Distribution: " + color.END)
    #print(pd.DataFrame(data=[W_test_norm[doc]], index = [doc], columns=categories+['general']))
    print_cumulative_train_doc_topics(data, doc_topic, doc, 11) 
    print()
    
    plot_radar_chart(doc_topic, doc)
    
def print_test_results(doc_topic, doc, corpus):
    print(color.BOLD + "Document " + str(doc) + color.END)
    print()
    print(color.BOLD + "Text: " + color.END)
    print("..." + corpus[doc][len(corpus[doc])//3:len(corpus[doc])//3+500] + "...")
    print()
    print()
    
    print(color.BOLD + "Topic Distribution: " + color.END)
    
    #print(pd.DataFrame(data=[W_test_norm[doc]], index = [doc], columns=categories+['general']))
    print_cumulative_test_doc_topics(doc_topic, doc, 11)
    print()
    
    plot_radar_chart(doc_topic, doc)
    
    

## Helper Functions

In [3]:
def cumulate_W(W, n_topics):
    W_cumul = []
    for d in W:
        temp = []
        for i in range(W.shape[1]//n_topics):
            temp.append(d[i*n_topics:(i+1)*n_topics].sum())
        W_cumul.append(temp)

    W_cumul = np.asarray(W_cumul)
    
    return W_cumul

def normalize_W(W):
    W_cumul_norm = W/(W.sum(axis=1).reshape(W.shape[0], 1))
    W_cumul_norm *= 100
    
    return W_cumul_norm

def export_to_excel(W, docs, filepath):
    '''
    Take cumulated W as input.
    Don't forget to put xlsx as file extension '''
    
    df = pd.DataFrame(data=W,index = range(len(W)), columns=categories+['general'])
    df['Text'] = docs
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    df.to_excel(filepath)
    return df

def export_to_csv(W, docs, filepath):
    '''
    Take cumulated W as input.
    Don't forget to put csv as file extension '''
    
    df = pd.DataFrame(data=W,index = range(len(W)), columns=categories+['general'])
    df['Text'] = docs
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    df.to_csv(filepath)
    return df

## Main Functions

In [4]:
def preprocess_corpus(corpus):
    PPcorpus = [' '.join(list((extract_terms(doc, extra_process = ['stem'])['Stem']+' ')*extract_terms(doc, extra_process = ['stem'])['TF'])) for doc in corpus]
    return PPcorpus
    
def evaluate_docs(docs, nmf, tfidf_vectorizer, betaloss = 'kullback-leibler'):
    print("Extracting tf-idf features for NMF...")
    t0 = time()
    tfidf_test = tfidf_vectorizer.transform(docs)
    #tfidf = tfidf_vectorizer.transform(corpusX)
    n_features = tfidf_test.shape[1]
    print("done in %0.2fs." % (time() - t0))
    
    X_test = tfidf_test
    H_test = nmf.components_
    
    
    # Fit the NMF model
    print("Fitting the NMF model (" + betaloss + ") with tf-idf features, ")
    t0 = time()

    W_test = nmf.transform(X_test)
    print("done in %0.2fs." % (time() - t0))
    
    return W_test, tfidf_test

In [5]:
def print_training_topics(pretrained_filepath):
    nmf, tfidf_vectorizer = pickle.load( open( pretrained_filepath, "rb" ) )
    print("\nTopics in NMF model:")
    print_top_words(nmf, tfidf_vectorizer, n_top_words=5, n_topics=3)

def add_corpus_txt(filepath, test_corpus):
    f = open(filepath, "r")
    txt = f.read()
    test_corpus.append(txt)
    f.close()
    
def add_corpus_url(url, api_key, test_corpus):
    insightIP = 'http://178.62.229.16'
    insightPort = '8484'
    insightVersion = 'v1.0'

    insightSetting = insightIP + ':' + insightPort + '/api/' + insightVersion 
    request = '/text_analytics/url_scraper?' + 'url=' + url + '&' + 'api_key=' + api_key

    # send a request
    res = requests.get(insightSetting + request)
    test_corpus.append(res.json()['text'])
    
def evaluate_test_corpus(pretrained_filepath, test_corpus):
    nmf, tfidf_vectorizer = pickle.load( open( pretrained_filepath, "rb" ) )
    test_corpusPP = preprocess_corpus(test_corpus)
    print()
    print('-'*30)
    print()
    W_test, tfidf_test = evaluate_docs(test_corpusPP, nmf, tfidf_vectorizer, betaloss = 'kullback-leibler')
    W_test_cumul = cumulate_W(W_test, n_topics=3)
    W_test_norm = normalize_W(W_test_cumul)
    
    return W_test_norm

def print_interactive_test_results(W_test_norm, test_corpus):
    interact(print_test_results, doc_topic=fixed(W_test_norm), doc = (0, len(W_test_norm)-1, 1), corpus=fixed(test_corpus))

## Print Pretrained Model's Topics

**nmf_pretrained.p** or **nmf_pretrained_pruned.p** includes pretrained NMF model generated using **Semi-Supervised-NMF-train.ipynb** notebook. It has the nmf model and tfidf_vectorizer.

for the details of purned version see also **"OMTermz HZ.ipynb"**

In [11]:
pre_trained_doc = "nmf_pretrained_pruned.p"
print_training_topics(pre_trained_doc)


Topics in NMF model:
universalism
Topic #0: individu topic analyt found help
Topic #1: peopl buell particl feder intern
Topic #2: disarma renew idol delight habitat

hedonism
Topic #3: time reaction simpli import ohatsu
Topic #4: studi shock see psycholog research
Topic #5: repres problem success induc scopophobia

achievement
Topic #6: theori peopl relat lower top
Topic #7: tribe humanist motiv introduc practic
Topic #8: social role merchant term other

power
Topic #9: articl lower may specialti peopl
Topic #10: partner idea bia belong use
Topic #11: leadership tool toxic environ guid

self-direction
Topic #12: use made gener liberti ratifi
Topic #13: known romantic resourc domin resent
Topic #14: benedek take olivero carrol interperson

benevolence
Topic #15: sometim seem natur one turkey
Topic #16: thought automat upon realiti answer
Topic #17: theori need renew shuv exampl

conformity
Topic #18: show thu uncodifi collectivist paint
Topic #19: other great rewritten problem use
Topi

## Evaluating Different Documents

Adding two example documents to the test_corpus.

In [7]:
test_corpus = []
add_corpus_txt("pope.txt", test_corpus) #Pope ted talk, https://www.ted.com/speakers/pope_francis
add_corpus_txt("dod.txt", test_corpus) # US Department of Defense, https://www.defense.gov/About/

Crawling a website using InSight API and adding its text to test_corpus.

Always check the text, added to the corpus via add_corpus_url. Because websites can have unexpected embedded texts.

In [8]:
insigth_api_key = "" #needs to be filled
url = "https://www.nationalgeographic.com/science/space/solar-system/earth/"
add_corpus_url(url, insigth_api_key, test_corpus)

Evaluate model for the test_corpus.

In [12]:
W_test_norm = evaluate_test_corpus(pre_trained_doc, test_corpus)

Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom stopword list is given, nltk.corpus.stopwords will be used.
File access error at ./data/stopwords_openmaker.txt loading is skipped.
File access error at ./data/specifics_openmaker.txt, data loading is skipped.
A single text is provided.
Extracting the terms ...
Tokenizing the input text ..
Done. Number of terms: 1857
Cleaning process: Initial size of tokens = 1857
Reduction due to punctuations and stopwords = 1332.
Reduction due to all numeral terms = 0
Reduction due to short terms = 0
Reduction due to rare terms = 0
Reduction due to partially numeral terms = 0
Reduction due to terms with not allowed symbols = 0
The total term count reduction during this cleaning process = 1332
Percentage = 72%
Stemming the terms in the corpus ..
Done.
COMPLETED.
Outputs will be written under D:\Boun\OpenMaker\Insight\semi-supervised-nmf/
Configuring the text cleaner ...
No custom sto

Results for test_corpus

In [13]:
print_interactive_test_results(W_test_norm, test_corpus)

interactive(children=(IntSlider(value=1, description='doc', max=2), Output()), _dom_classes=('widget-interact',))

In [14]:
df = export_to_excel(W_test_norm, test_corpus, filepath = 'output.xlsx')
df.head()

,Text,universalism,hedonism,achievement,power,self-direction,benevolence,conformity,tradition,stimulation,security,general
0,"Good evening â€“ or, good morning, I am not su...",4.024334,8.704034,6.412505,9.568149,2.022214,15.938469,31.645763,5.662532,5.513720,5.292175,5.216106
1,\nOn behalf of the Secretary of Defense and De...,40.971534,1.119135,3.567248,5.178333,6.909340,0.000029,3.523228,0.025827,0.498702,28.338515,9.868110
2,"Earth, our home planet, is the only planet in ...",21.073571,1.817401,0.093295,15.418328,5.277123,1.962361,2.451394,0.019308,48.444014,2.293843,1.149362


In [15]:
df = export_to_csv(W_test_norm, test_corpus, filepath = 'output.csv')
df.head()

,Text,universalism,hedonism,achievement,power,self-direction,benevolence,conformity,tradition,stimulation,security,general
0,"Good evening â€“ or, good morning, I am not su...",4.024334,8.704034,6.412505,9.568149,2.022214,15.938469,31.645763,5.662532,5.513720,5.292175,5.216106
1,\nOn behalf of the Secretary of Defense and De...,40.971534,1.119135,3.567248,5.178333,6.909340,0.000029,3.523228,0.025827,0.498702,28.338515,9.868110
2,"Earth, our home planet, is the only planet in ...",21.073571,1.817401,0.093295,15.418328,5.277123,1.962361,2.451394,0.019308,48.444014,2.293843,1.149362
